<a href="https://colab.research.google.com/github/Jcain17/Intro_to_ML/blob/main/ML_HW6_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
%matplotlib inline
import torch
import numpy as np
import torch.optim as optim
from google.colab import drive
import pandas as pd
from sklearn.preprocessing import StandardScaler
import torch.nn as nn
from collections import OrderedDict

In [ ]:
drive.mount('/content/drive')

file_path = '/content/drive/My Drive/Housing.csv'
housing = pd.DataFrame(pd.read_csv(file_path))
housing.head()

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


,price,area,bedrooms,bathrooms,stories,mainroad,guestroom,basement,hotwaterheating,airconditioning,parking,prefarea,furnishingstatus
0,13300000,7420,4,2,3,yes,no,no,no,yes,2,yes,furnished
1,12250000,8960,4,4,4,yes,no,no,no,yes,3,no,furnished
2,12250000,9960,3,2,2,yes,no,yes,no,no,2,yes,semi-furnished
3,12215000,7500,4,2,2,yes,no,yes,no,yes,3,yes,furnished
4,11410000,7420,4,1,2,yes,yes,yes,no,yes,2,no,furnished


In [ ]:
varlist2 = ['mainroad', 'guestroom', 'basement', 'hotwaterheating', 'airconditioning', 'prefarea']

def binmap(x):
   return x.map({'yes': 1, 'no': 0})

housing[varlist2] = housing[varlist2].apply(binmap)
housing.head()

,price,area,bedrooms,bathrooms,stories,mainroad,guestroom,basement,hotwaterheating,airconditioning,parking,prefarea,furnishingstatus
0,13300000,7420,4,2,3,1,0,0,0,1,2,1,furnished
1,12250000,8960,4,4,4,1,0,0,0,1,3,0,furnished
2,12250000,9960,3,2,2,1,0,1,0,0,2,1,semi-furnished
3,12215000,7500,4,2,2,1,0,1,0,1,3,1,furnished
4,11410000,7420,4,1,2,1,1,1,0,1,2,0,furnished


In [ ]:
sc_X = StandardScaler()
varlist1 = ['price', 'area', 'bedrooms', 'bathrooms', 'stories', 'parking', 'mainroad', 'guestroom', 'basement', 'hotwaterheating', 'airconditioning', 'prefarea']
housing[varlist1] = sc_X.fit_transform(housing[varlist1])

varlist = ['area', 'bedrooms', 'bathrooms', 'stories', 'parking', 'mainroad', 'guestroom', 'basement', 'hotwaterheating', 'airconditioning', 'prefarea']

X = housing[varlist].values
Y = housing['price']

X,Y

(array([[ 1.04672629,  1.40341936,  1.42181174, ..., -0.2192645 ,
          1.4726183 ,  1.80494113],
        [ 1.75700953,  1.40341936,  5.40580863, ..., -0.2192645 ,
          1.4726183 , -0.55403469],
        [ 2.21823241,  0.04727831,  1.42181174, ..., -0.2192645 ,
         -0.67906259,  1.80494113],
        ...,
        [-0.70592066, -1.30886273, -0.57018671, ..., -0.2192645 ,
         -0.67906259, -0.55403469],
        [-1.03338891,  0.04727831, -0.57018671, ..., -0.2192645 ,
         -0.67906259, -0.55403469],
        [-0.5998394 ,  0.04727831, -0.57018671, ..., -0.2192645 ,
         -0.67906259, -0.55403469]]),
 0      4.566365
 1      4.004484
 2      4.004484
 3      3.985755
 4      3.554979
          ...   
 540   -1.576868
 541   -1.605149
 542   -1.614327
 543   -1.614327
 544   -1.614327
 Name: price, Length: 545, dtype: float64)

In [ ]:
x = torch.tensor(X, dtype = torch.float32)
y = torch.tensor(Y, dtype = torch.float32)
x.size(),y.size()

(torch.Size([545, 11]), torch.Size([545]))

In [ ]:
n_samples = x.shape[0]
n_val = int(0.2 * n_samples)

shuffled_indices = torch.randperm(n_samples)

train_indices = shuffled_indices[:-n_val]
val_indices = shuffled_indices[-n_val:]

train_x = x[train_indices]

val_x = x[val_indices]

train_xu = x[train_indices]

val_xu = x[val_indices]

train_y = y[train_indices]

val_y = y[val_indices]

train_indices.size(), val_indices.size()

(torch.Size([436]), torch.Size([109]))

In [ ]:
def training_loop(n_epochs, optimizer, model, loss_fn, t_u_train, t_u_val,
                  t_c_train, t_c_val):
    for epoch in range(1, n_epochs + 1):
        t_p_train = model(t_u_train) # <1>
        loss_train = loss_fn(t_p_train, t_c_train)

        t_p_val = model(t_u_val) # <1>
        loss_val = loss_fn(t_p_val, t_c_val)

        optimizer.zero_grad()
        loss_train.backward() # <2>
        optimizer.step()

        if epoch == 1 or epoch % 1000 == 0:
            print(f"Epoch {epoch}, Training loss {loss_train.item():.4f},"
                  f" Validation loss {loss_val.item():.4f}")

In [ ]:
seq_model = nn.Sequential(nn.Linear(11,32), nn.Tanh(), nn.Linear(32,1))
seq_model

Sequential(
  (0): Linear(in_features=11, out_features=32, bias=True)
  (1): Tanh()
  (2): Linear(in_features=32, out_features=1, bias=True)
)

In [ ]:
[param.shape for param in seq_model.parameters()]

[torch.Size([32, 11]), torch.Size([32]), torch.Size([1, 32]), torch.Size([1])]

In [ ]:
for name, param in seq_model.named_parameters():
    print(name, param.shape)

0.weight torch.Size([32, 11])
0.bias torch.Size([32])
2.weight torch.Size([1, 32])
2.bias torch.Size([1])


In [ ]:
seq_model = nn.Sequential(OrderedDict([('hidden_linear', nn.Linear(11,32)), ('hidden_activation', nn.Tanh()), ('output_linear', nn.Linear(32,1))]))

seq_model

Sequential(
  (hidden_linear): Linear(in_features=11, out_features=32, bias=True)
  (hidden_activation): Tanh()
  (output_linear): Linear(in_features=32, out_features=1, bias=True)
)

In [ ]:
for name, param in seq_model.named_parameters():
    print(name, param.shape)

hidden_linear.weight torch.Size([32, 11])
hidden_linear.bias torch.Size([32])
output_linear.weight torch.Size([1, 32])
output_linear.bias torch.Size([1])


In [ ]:
seq_model.output_linear.bias

Parameter containing:
tensor([-0.0843], requires_grad=True)

In [ ]:
optimizer = optim.SGD(seq_model.parameters(), lr=1e-3)

training_loop(
    n_epochs = 5000,
    optimizer = optimizer,
    model = seq_model,
    loss_fn = nn.MSELoss(),
    t_u_train = train_xu,
    t_u_val = val_xu,
    t_c_train = train_y,
    t_c_val = val_y)

print('output', seq_model(val_xu))
print('answer', val_x)
print('hidden', seq_model.hidden_linear.weight.grad)

Epoch 1, Training loss 1.1656, Validation loss 0.7699


/usr/local/lib/python3.10/dist-packages/torch/nn/modules/loss.py:535: UserWarning: Using a target size (torch.Size([436])) that is different to the input size (torch.Size([436, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)
/usr/local/lib/python3.10/dist-packages/torch/nn/modules/loss.py:535: UserWarning: Using a target size (torch.Size([109])) that is different to the input size (torch.Size([109, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)


Epoch 1000, Training loss 1.0721, Validation loss 0.7480
Epoch 2000, Training loss 1.0673, Validation loss 0.7468
Epoch 3000, Training loss 1.0660, Validation loss 0.7464
Epoch 4000, Training loss 1.0655, Validation loss 0.7464
Epoch 5000, Training loss 1.0654, Validation loss 0.7465
output tensor([[ 4.4738e-02],
        [ 4.1622e-02],
        [ 1.9975e-02],
        [-8.2859e-03],
        [-9.9336e-03],
        [ 2.0892e-02],
        [ 2.3934e-02],
        [ 5.2926e-02],
        [-4.1309e-03],
        [ 4.2249e-02],
        [ 5.2212e-02],
        [ 2.1928e-02],
        [ 6.5399e-02],
        [ 1.4465e-02],
        [-6.1400e-03],
        [ 1.4165e-02],
        [ 4.0743e-02],
        [ 3.6143e-02],
        [ 1.7845e-02],
        [ 2.5805e-02],
        [ 4.1148e-02],
        [ 6.9866e-02],
        [ 4.8599e-02],
        [ 3.1838e-02],
        [ 1.9696e-02],
        [ 4.4096e-02],
        [ 3.0151e-02],
        [ 5.4090e-02],
        [ 4.6046e-02],
        [ 3.7068e-02],
        [ 1.9720e-

In [ ]:
seq_model = nn.Sequential(OrderedDict([('hidden_linear1', nn.Linear(11,32)),
  ('hidden_activation1', nn.Tanh()),
  ('hidden_linear2', nn.Linear(32,64)),
  ('hidden_activation2', nn.Tanh()),
  ('hidden_linea3', nn.Linear(64,16)),
  ('hidden_activation3', nn.Tanh()),
  ('output_linear', nn.Linear(16,1))]))

seq_model

Sequential(
  (hidden_linear1): Linear(in_features=11, out_features=32, bias=True)
  (hidden_activation1): Tanh()
  (hidden_linear2): Linear(in_features=32, out_features=64, bias=True)
  (hidden_activation2): Tanh()
  (hidden_linea3): Linear(in_features=64, out_features=16, bias=True)
  (hidden_activation3): Tanh()
  (output_linear): Linear(in_features=16, out_features=1, bias=True)
)

In [ ]:
for name, param in seq_model.named_parameters():
    print(name, param.shape)

hidden_linear1.weight torch.Size([32, 11])
hidden_linear1.bias torch.Size([32])
hidden_linear2.weight torch.Size([64, 32])
hidden_linear2.bias torch.Size([64])
hidden_linea3.weight torch.Size([16, 64])
hidden_linea3.bias torch.Size([16])
output_linear.weight torch.Size([1, 16])
output_linear.bias torch.Size([1])


In [ ]:
seq_model.output_linear.bias

Parameter containing:
tensor([-0.1393], requires_grad=True)

In [ ]:
optimizer = optim.SGD(seq_model.parameters(), lr=1e-3)

training_loop(
    n_epochs = 5000,
    optimizer = optimizer,
    model = seq_model,
    loss_fn = nn.MSELoss(),
    t_u_train = train_xu,
    t_u_val = val_xu,
    t_c_train = train_y,
    t_c_val = val_y)

print('output', seq_model(val_xu))
print('answer', val_x)

Epoch 1, Training loss 1.0747, Validation loss 0.7415
Epoch 1000, Training loss 1.0681, Validation loss 0.7473
Epoch 2000, Training loss 1.0668, Validation loss 0.7469
Epoch 3000, Training loss 1.0662, Validation loss 0.7468
Epoch 4000, Training loss 1.0659, Validation loss 0.7468
Epoch 5000, Training loss 1.0657, Validation loss 0.7467
output tensor([[ 0.0119],
        [ 0.0141],
        [ 0.0204],
        [ 0.0017],
        [ 0.0865],
        [ 0.0409],
        [ 0.0467],
        [ 0.0349],
        [ 0.0126],
        [-0.0153],
        [ 0.0668],
        [ 0.0111],
        [ 0.0302],
        [ 0.0783],
        [ 0.0812],
        [ 0.0283],
        [ 0.0280],
        [ 0.0239],
        [ 0.0390],
        [ 0.0890],
        [ 0.0399],
        [ 0.0680],
        [-0.0153],
        [-0.0163],
        [ 0.0751],
        [ 0.0448],
        [ 0.0454],
        [ 0.0540],
        [ 0.0139],
        [-0.0069],
        [ 0.0209],
        [ 0.0873],
        [ 0.0583],
        [ 0.0271],
        